In [1]:
import sqlite3
import json
from datetime import datetime

In [2]:
timeframe = '2015-01'
sql_transaction = []

conn = sqlite3.connect('{}.db'.format(timeframe))
cur = conn.cursor()

In [3]:
def create_table():
    cur.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

In [4]:
create_table()

In [5]:
row_counter = 0
paired_rows = 0

In [6]:
def format_data(data):
    data = data.replace('\n', ' newlinechar ').replace('\r', ' newlinechar ').replace('"', "'")
    return data

In [7]:
def find_parent(pid):
    try:
        sql = 'select comment from parent_reply where comment_id = "{}" limit 1'.format(pid)
        cur.execute(sql)
        result = cur.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        print("find_parent", e)
        return False

In [8]:
def find_existing_score(pid):
    try:
        sql = 'select score from parent_reply where parent_id = "{}" limit 1'.format(pid)
        cur.execute(sql)
        result = cur.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        print("find_parent_score", e)
        return False

In [9]:
def acceptable(data):
    if len(data.split(' ')) > 50  or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True
        
    

In [10]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        cur.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                cur.execute(s)
            except:
                pass
        conn.commit()
        sql_transaction = []

In [11]:
def sql_insert_replace_comment(commentid,parentid, parent, comment, subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
        
    except Exception as e:
        print('s0 insertion', str(e))

In [12]:
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-PARENT insertion',str(e))




In [13]:
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-NO_PARENT insertion',str(e))

In [15]:
with open('RC_2015-01.json', buffering=1000) as f:
    for row in f:
        row_counter += 1
        row = json.loads(row)
        parent_id = row['parent_id']
        body = format_data(row['body'])
        created_utc = row['created_utc']
        score = row['score']
        comment_id = row['name']
        subreddit = row['subreddit']
        parent_data = find_parent(parent_id)
        if score >= 2:
            existing_comment_score = find_existing_score(parent_id)
            if existing_comment_score:
                if score > existing_comment_score:
                    if acceptable(body):
                        sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)

            else:
                if acceptable(body):
                    if parent_data:
                        sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                        paired_rows += 1
                    else:
                        sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                            
        if row_counter % 100000 == 0:
            print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))
        

Total Rows Read: 200000, Paired Rows: 5260, Time: 2017-11-29 20:01:21.448648
Total Rows Read: 300000, Paired Rows: 6843, Time: 2017-11-29 20:02:00.291825
Total Rows Read: 400000, Paired Rows: 12077, Time: 2017-11-29 20:03:54.021570
Total Rows Read: 500000, Paired Rows: 17426, Time: 2017-11-29 20:05:39.692002
Total Rows Read: 600000, Paired Rows: 22691, Time: 2017-11-29 20:07:37.053514
Total Rows Read: 700000, Paired Rows: 27368, Time: 2017-11-29 20:10:04.336053
Total Rows Read: 800000, Paired Rows: 31894, Time: 2017-11-29 20:11:57.208449
Total Rows Read: 900000, Paired Rows: 37162, Time: 2017-11-29 20:13:51.905734
Total Rows Read: 1000000, Paired Rows: 42853, Time: 2017-11-29 20:15:41.826213
Total Rows Read: 1100000, Paired Rows: 48398, Time: 2017-11-29 20:17:25.721848
Total Rows Read: 1200000, Paired Rows: 54028, Time: 2017-11-29 20:18:56.149047
Total Rows Read: 1300000, Paired Rows: 59605, Time: 2017-11-29 20:20:30.998344
Total Rows Read: 1400000, Paired Rows: 65246, Time: 2017-11-29

Total Rows Read: 10500000, Paired Rows: 580189, Time: 2017-11-29 22:13:32.862802
Total Rows Read: 10600000, Paired Rows: 585666, Time: 2017-11-29 22:14:39.034013
Total Rows Read: 10700000, Paired Rows: 590643, Time: 2017-11-29 22:15:47.157879
Total Rows Read: 10800000, Paired Rows: 595553, Time: 2017-11-29 22:16:48.146284
Total Rows Read: 10900000, Paired Rows: 600836, Time: 2017-11-29 22:17:49.002543
Total Rows Read: 11000000, Paired Rows: 606448, Time: 2017-11-29 22:18:52.207589
Total Rows Read: 11100000, Paired Rows: 612385, Time: 2017-11-29 22:19:55.834748
Total Rows Read: 11200000, Paired Rows: 618497, Time: 2017-11-29 22:21:05.623640
Total Rows Read: 11300000, Paired Rows: 624392, Time: 2017-11-29 22:22:10.348319
Total Rows Read: 11400000, Paired Rows: 630520, Time: 2017-11-29 22:23:17.410968
Total Rows Read: 11500000, Paired Rows: 636708, Time: 2017-11-29 22:24:21.757959
Total Rows Read: 11600000, Paired Rows: 642887, Time: 2017-11-29 22:25:29.833687
Total Rows Read: 11700000, P

Total Rows Read: 20600000, Paired Rows: 1158617, Time: 2017-11-30 00:19:44.590017
Total Rows Read: 20700000, Paired Rows: 1163883, Time: 2017-11-30 00:20:47.980232
Total Rows Read: 20800000, Paired Rows: 1169311, Time: 2017-11-30 00:22:02.464782
Total Rows Read: 20900000, Paired Rows: 1174542, Time: 2017-11-30 00:23:12.493517
Total Rows Read: 21000000, Paired Rows: 1179921, Time: 2017-11-30 00:24:23.852829
Total Rows Read: 21100000, Paired Rows: 1185340, Time: 2017-11-30 00:25:33.107944
Total Rows Read: 21200000, Paired Rows: 1189947, Time: 2017-11-30 00:26:32.917409
Total Rows Read: 21300000, Paired Rows: 1194853, Time: 2017-11-30 00:27:35.512074
Total Rows Read: 21400000, Paired Rows: 1200280, Time: 2017-11-30 00:28:36.151591
Total Rows Read: 21500000, Paired Rows: 1206103, Time: 2017-11-30 00:29:39.470472
Total Rows Read: 21600000, Paired Rows: 1212054, Time: 2017-11-30 00:30:44.545443
Total Rows Read: 21700000, Paired Rows: 1218005, Time: 2017-11-30 00:31:45.552865
Total Rows Read:

Total Rows Read: 30600000, Paired Rows: 1729053, Time: 2017-11-30 02:11:07.892319
Total Rows Read: 30700000, Paired Rows: 1734079, Time: 2017-11-30 02:12:04.695375
Total Rows Read: 30800000, Paired Rows: 1738616, Time: 2017-11-30 02:12:57.906168
Total Rows Read: 30900000, Paired Rows: 1743478, Time: 2017-11-30 02:13:52.753291
Total Rows Read: 31000000, Paired Rows: 1748965, Time: 2017-11-30 02:14:49.878061
Total Rows Read: 31100000, Paired Rows: 1754378, Time: 2017-11-30 02:15:51.644938
Total Rows Read: 31200000, Paired Rows: 1760244, Time: 2017-11-30 02:16:53.894055
Total Rows Read: 31300000, Paired Rows: 1766177, Time: 2017-11-30 02:17:59.981326
Total Rows Read: 31400000, Paired Rows: 1772082, Time: 2017-11-30 02:19:05.929122
Total Rows Read: 31500000, Paired Rows: 1777772, Time: 2017-11-30 02:20:10.958709
Total Rows Read: 31600000, Paired Rows: 1782727, Time: 2017-11-30 02:21:16.269330
Total Rows Read: 31700000, Paired Rows: 1787730, Time: 2017-11-30 02:22:21.785044
Total Rows Read:

Total Rows Read: 40600000, Paired Rows: 2297229, Time: 2017-11-30 04:00:41.712056
Total Rows Read: 40700000, Paired Rows: 2302497, Time: 2017-11-30 04:01:46.243281
Total Rows Read: 40800000, Paired Rows: 2307393, Time: 2017-11-30 04:02:52.838837
Total Rows Read: 40900000, Paired Rows: 2312399, Time: 2017-11-30 04:03:52.928411
Total Rows Read: 41000000, Paired Rows: 2318044, Time: 2017-11-30 04:04:54.789395
Total Rows Read: 41100000, Paired Rows: 2323662, Time: 2017-11-30 04:06:00.088474
Total Rows Read: 41200000, Paired Rows: 2329644, Time: 2017-11-30 04:07:03.511341
Total Rows Read: 41300000, Paired Rows: 2335668, Time: 2017-11-30 04:08:09.768725
Total Rows Read: 41400000, Paired Rows: 2341691, Time: 2017-11-30 04:09:18.117406
Total Rows Read: 41500000, Paired Rows: 2347662, Time: 2017-11-30 04:10:37.181234
Total Rows Read: 41600000, Paired Rows: 2353511, Time: 2017-11-30 04:11:46.665509
Total Rows Read: 41700000, Paired Rows: 2359102, Time: 2017-11-30 04:12:56.728707
Total Rows Read:

Total Rows Read: 50600000, Paired Rows: 2872127, Time: 2017-11-30 06:01:01.558445
Total Rows Read: 50700000, Paired Rows: 2878083, Time: 2017-11-30 06:02:16.718434
Total Rows Read: 50800000, Paired Rows: 2883898, Time: 2017-11-30 06:03:35.647090
Total Rows Read: 50900000, Paired Rows: 2889708, Time: 2017-11-30 06:05:16.054688
Total Rows Read: 51000000, Paired Rows: 2895800, Time: 2017-11-30 06:06:50.473966
Total Rows Read: 51100000, Paired Rows: 2901643, Time: 2017-11-30 06:08:27.457736
Total Rows Read: 51200000, Paired Rows: 2907129, Time: 2017-11-30 06:09:56.350859
Total Rows Read: 51300000, Paired Rows: 2912168, Time: 2017-11-30 06:11:30.502052
Total Rows Read: 51400000, Paired Rows: 2917093, Time: 2017-11-30 06:12:50.095974
Total Rows Read: 51500000, Paired Rows: 2922589, Time: 2017-11-30 06:14:08.289177
Total Rows Read: 51600000, Paired Rows: 2928363, Time: 2017-11-30 06:15:21.502240
Total Rows Read: 51700000, Paired Rows: 2934219, Time: 2017-11-30 06:16:39.200560
Total Rows Read: